In [10]:
import time
import datetime
import pandas
import numpy as np
import random
# from yahoo_finance import Shareb
from pandas.tseries.offsets import BDay

In [11]:
from bokeh.io import output_notebook, show, output_file,save
output_notebook()

Loading BokehJS ...

User will:

Input dollar amount to invest in USD (Minimum is $5000 USD)
Pick one or two investment strategies:


#Ethical Investing
TSLA, PBW, SCTY, AAPL
#Growth Investing
CRM, TWLO, NVDA
#Index Investing
SPY, VTI, VWO , VBR
#Quality Investing
FB, XOM, COST, HD
#Value Investing
AMZN, MSFT, NEFLX, DIS

In [12]:
#initiate the mapping for different strategies
strategies = {'ethical':['TSLA','PBW','SCTY','AAPL'], 'growth':['CRM','TWLO','NVDA'], 'index':['SPY','VTI','VWO','VBR'], 'quality':['FB', 'XOM','COST','HD'],'value':['AMZN','MSFT','NFLX','DIS']}

In [13]:
choice = random.choice(['ethical','growth','index','quality','value'])

In [14]:
#total amount to invest
total = 6000
stockslist = strategies[choice]
singlestock = total/len(stockslist)

In [15]:
enddate = datetime.datetime.now().date()- datetime.timedelta(1)
startdate = enddate - BDay(4)

In [16]:
from pandas_datareader import data, wb
import pandas_datareader.data as web

In [17]:
stockslist

['FB', 'XOM', 'COST', 'HD']

In [18]:
#getting historical data
df = web.DataReader(stockslist,'yahoo',enddate - BDay(4), enddate)

In [19]:
#getting today's quote
df_today=web.get_quote_yahoo(stockslist)

In [20]:
df_historical = df.to_frame()

In [24]:
#getting the name of the sticker
def getname(li):
    df = []
    for i in li:
        df.append(web.get_components_yahoo(i))
    return pd.concat(df)


In [25]:
df_info = getname(stockslist)

#checking dataframes df_info, df_historical, df_today

In [26]:
df_info

,name,exchange
ticker,,
FB,"Facebook, Inc.",NMS
XOM,Exxon Mobil Corporation Common,NYQ
COST,Costco Wholesale Corporation,NMS
HD,"Home Depot, Inc. (The) Common S",NYQ


In [27]:
df_today

,PE,change_pct,last,short_ratio,time
FB,55.94,-0.65%,117.02,1.38,4:00pm
XOM,39.96,+0.06%,85.28,4.25,4:00pm
COST,28.21,-0.56%,150.36,3.44,4:00pm
HD,21.72,-0.47%,128.33,1.60,4:01pm


In [28]:
df_historical

Open        High         Low       Close    Volume  \
Date       minor                                                             
2016-11-11 COST   146.610001  149.619995  145.380005  149.360001   3218000   
           FB     119.529999  120.699997  118.150002  119.019997  32804000   
           HD     129.059998  130.130005  128.720001  129.850006   5490000   
           XOM     86.529999   86.730003   84.889999   85.669998  13711000   
2016-11-14 COST   150.919998  152.139999  149.500000  152.000000   3890800   
           FB     119.129997  119.129997  113.550003  115.080002  51247400   
           HD     130.500000  130.639999  126.940002  127.669998  11405400   
           XOM     85.300003   85.650002   84.330002   85.279999  12553200   
2016-11-15 COST   151.979996  152.000000  150.289993  151.149994   2721800   
           FB     116.730003  118.489998  115.830002  117.199997  33270700   
           HD     127.070000  127.669998  122.919998  124.400002  13423400   
           XOM     85.769997   87.059998   85.769997   86.820000  12806500   
2016-11-16 COST   152.190002  152.550003  151.800003  152.139999   2151200   
           FB     114.480003  117.879997  114.209999  116.339996  32345400   
           HD     123.599998  125.699997  123.279999  125.330002   9767500   
           XOM     86.739998   86.779999   85.580002   85.750000   8119400   
2016-11-17 COST   151.229996  151.399994  149.550003  151.199997   2096800   
           FB     116.809998  117.790001  116.010002  117.790001  18556200   
           HD     125.870003  129.139999  125.699997  128.929993   6727500   
           XOM     86.440002   86.879997   85.019997   85.230003   8147600   

                   Adj Close  
Date       minor              
2016-11-11 COST   149.360001  
           FB     119.019997  
           HD     129.850006  
           XOM     85.669998  
2016-11-14 COST   152.000000  
           FB     115.080002  
           HD     127.669998  
           XOM     85.279999  
2016-11-15 COST   151.149994  
           FB     117.199997  
           HD     124.400002  
           XOM     86.820000  
2016-11-16 COST   152.139999  
           FB     116.339996  
           HD     125.330002  
           XOM     85.750000  
2016-11-17 COST   151.199997  
           FB     117.790001  
           HD     128.929993  
           XOM     85.230003

Calculating the portfolio values


In [29]:
df_historical.loc[str(startdate)]['Close']

minor
COST     149.360001
FB       119.019997
HD       129.850006
XOM       85.669998
Name: Close, dtype: float64

In [30]:
#calculating shares acquired on Day1
def getnumshares(li, date, df_historical, singlestock):
    numshares={}
    for i in li:
        numshares[i] = singlestock/df_historical.loc[date]['Close'][i]
    return numshares

In [31]:
num_shares=getnumshares(stockslist, str(startdate),df_historical, singlestock)

In [32]:
def getportfolio_value(numshares, stocklist, date, df_historical):
    portfolio_value = 0
    for i in stocklist:
        portfolio_value += numshares[i] * df_historical.loc[date]['Close'][i]
    return portfolio_value

In [33]:
getportfolio_value(num_shares, stockslist, startdate,df_historical)

6000.0

In [34]:
getportfolio_value(num_shares, stockslist, startdate+ BDay(2),df_historical)

5952.2174452388745

In [35]:
#build a dateframe to track historical portfolio value change
def track_portfolio_value(startdate, delta, stocklist, df_historical,numshares):
    portfolio_values={}
    for i in range(delta):
        date = startdate + BDay(i)
        portfolio_values[str(date)] = getportfolio_value(numshares, stocklist, date, df_historical)
    return portfolio_values

In [36]:
d = track_portfolio_value(startdate,5,stockslist, df_historical, num_shares)

In [37]:
df_values = pd.DataFrame(d.items(), columns=['Date','totalvalue'])

In [57]:
df_values.Date = pd.to_datetime(df_values.Date)

In [58]:
df_values

,Date,totalvalue
0,2016-11-11,6000.000000
1,2016-11-14,5944.846149
2,2016-11-15,5952.217445
3,2016-11-16,5943.329874
4,2016-11-17,5984.645566


#building charts


In [79]:
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter

In [70]:
df_values.describe()

,totalvalue
count,5.000000
mean,5965.007807
std,25.739628
min,5943.329874
25%,5944.846149
50%,5952.217445
75%,5984.645566
max,6000.000000


In [83]:
max = df_values.totalvalue.max()+ df_values.totalvalue.std()
min = df_values.totalvalue.min()-df_values.totalvalue.std()
p = figure(plot_width = 500, plot_height = 300,x_axis_type="datetime",y_range=(min,max), title="Portfolio Total Value trend")
p.line(df_values['Date'], df_values['totalvalue'], line_width =3)
p.yaxis[0].formatter = NumeralTickFormatter(format="$0.00")
p.xaxis.axis_label='Date'
p.yaxis.axis_label='Total value'
show(p)

In [86]:
df_historical_chart = df_historical.reset_index()
df_historical_chart

,Date,minor,Open,High,Low,Close,Volume,Adj Close
0,2016-11-11,COST,146.610001,149.619995,145.380005,149.360001,3218000,149.360001
1,2016-11-11,FB,119.529999,120.699997,118.150002,119.019997,32804000,119.019997
2,2016-11-11,HD,129.059998,130.130005,128.720001,129.850006,5490000,129.850006
3,2016-11-11,XOM,86.529999,86.730003,84.889999,85.669998,13711000,85.669998
4,2016-11-14,COST,150.919998,152.139999,149.500000,152.000000,3890800,152.000000
5,2016-11-14,FB,119.129997,119.129997,113.550003,115.080002,51247400,115.080002
6,2016-11-14,HD,130.500000,130.639999,126.940002,127.669998,11405400,127.669998
7,2016-11-14,XOM,85.300003,85.650002,84.330002,85.279999,12553200,85.279999
8,2016-11-15,COST,151.979996,152.000000,150.289993,151.149994,2721800,151.149994
9,2016-11-15,FB,116.730003,118.489998,115.830002,117.199997,33270700,117.199997


In [94]:
df_chart=pd.pivot_table(df_historical_chart, values='Adj Close', columns='minor', index='Date')
df_chart.reset_index(inplace=True)
df_chart

minor,Date,COST,FB,HD,XOM
0,2016-11-11,149.360001,119.019997,129.850006,85.669998
1,2016-11-14,152.000000,115.080002,127.669998,85.279999
2,2016-11-15,151.149994,117.199997,124.400002,86.820000
3,2016-11-16,152.139999,116.339996,125.330002,85.750000
4,2016-11-17,151.199997,117.790001,128.929993,85.230003


In [95]:
stockslist

['FB', 'XOM', 'COST', 'HD']

In [100]:
from bokeh.charts import Line, show, output_file
p2 = figure(x_axis_type='datetime', title='Stock Closing Prices')
p2.xaxis.axis_label = 'Date'
p2.yaxis.axis_label = 'Price'

colors = ['#A6CEE3', '#99d594', '#e6f598', '#fee08b']
for i, j in zip(stockslist, colors):
    p2.line(df_chart.Date, df_chart[i], legend=i, color=j, line_width=5)

show(p2)